In [ ]:
import matplotlib.pyplot as plt
import ipyplot
import numpy as np
import cv2
import math
import statistics
from scipy.signal import find_peaks
from collections import Counter

In [ ]:
img_orig = cv2.imread("jigsawsqr.png")
img_orig = img_orig[70:420, 75:440]
h, w = img_orig.shape[:2]
img_orig = cv2.resize(img_orig, (4*w, 4*h))
plt.imshow(img_orig);

In [ ]:
img_gray = cv2.cvtColor(img_orig, cv2.COLOR_BGR2GRAY)
plt.imshow(img_gray);

In [ ]:
img_edges = cv2.Canny(image=img_gray, threshold1=100, threshold2=200)
plt.imshow(img_edges);

In [ ]:
_, img_binary = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
raw_contours, _ = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
img = img_orig.copy()
cv2.drawContours(img, raw_contours, -1, (0, 255, 0), 2)
print("Number of detected contours: ", len(raw_contours))
plt.imshow(img);

In [ ]:
areas = [cv2.contourArea(contour) for contour in contours]
median_area = statistics.median(areas)
min_area = 0.5 * median_area
max_area = 2 * median_area
print(f"Ignore too big or small shapes, median_area: {median_area}")

contours = [contour for contour, area in zip(raw_contours, areas) if 0.5 < area / median_area < 2]
nb_pieces = len(contours)
print(f"Number of detected pieces: {nb_pieces}")

rects = [cv2.boundingRect(contour) for contour in contours]
img = img_orig.copy()
for (x, y, w, h) in rects:
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

plt.imshow(img);

In [ ]:
min_area_rects = []

img = img_orig.copy()
for contour in contours:
    rect = cv2.minAreaRect(contour)
    (cx, cy), (sx, sy), angle = rect
    if sy < sx:
        angle = (angle + 90) % 360
        sx, sy = sy, sx
    rect = ((cx, cy), (sx, sy), angle)
    min_area_rects.append(rect)

    box = np.int0(cv2.boxPoints(rect))
    cv2.drawContours(img, [box], 0, (0, 255, 0), 2)

plt.imshow(img);

In [ ]:
imgs = []
for contour, rect in zip(contours, rects):
    x, y, w, h = rect
    imgs.append(img_gray[y:y+h, x:x+w])

ipyplot.plot_images(imgs, img_width=75);

In [ ]:
polar_imgs = []
for contour, rect, min_area_rect in zip(contours, rects, min_area_rects):
    piece_degrees = min_area_rect[2]
    x, y, w, h = rect
    img = cv2.linearPolar(img_gray[y:y+h, x:x+w], (w/2, h/2), max(h, w), cv2.WARP_FILL_OUTLIERS)
    y0 = int(piece_degrees * h / 360) % 360
    img = np.concatenate([img[y0:, :], img[:y0, :]])
    _, img_binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    polar_imgs.append(img_binary)

ipyplot.plot_images(polar_imgs, img_width=75); 

In [ ]:
imgs = []
all_polar_peaks = []  # list of [(peak_radius, peak_degrees)]

for polar_img in polar_imgs:
    img_reversed = np.flip(polar_img, axis=1)
    h, w = polar_img.shape[:2]
    radius = w - np.argmax(img_reversed, axis=1)
    peak_indices = find_peaks(radius, prominence=5)[0]

    polar_peaks = [(radius[idx] * max(h, w) / w, idx * 360 / h) for idx in peak_indices]
    all_polar_peaks.append(polar_peaks)

    img = polar_img.copy()
    for idx in peak_indices:
        cv2.circle(img, (radius[idx], idx), 15, 180, 2)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=150);

In [ ]:
img = img_orig.copy()

all_corners = []  # list of [(x, y)]
for contour, rect, min_area_rect, polar_peaks in zip(contours, rects, min_area_rects, all_polar_peaks):
    piece_degrees = min_area_rect[2]
    x, y, w, h = rect
    cx = x + w/2
    cy = y + h/2

    for peak_radius, peak_degrees in polar_peaks:
        peak_radians = math.radians(peak_degrees + piece_degrees)
        dx = peak_radius * math.cos(peak_radians)
        dy = peak_radius * math.sin(peak_radians)
        cv2.circle(img, (int(cx + dx), int(cy + dy)), 20, (0, 200, 0), 2)

    peaks = {}  # key=quarter, value=[(abs(angle), angle, distance)] where angles are relative to the piece_angle
    for peak_radius, peak_degrees in polar_peaks:
        quarter = peak_degrees // 90
        diff_degrees = peak_degrees % 180
        if diff_degrees > 90:
            diff_degrees -= 180
        peaks.setdefault(quarter, []).append((abs(diff_degrees), peak_radius, peak_degrees + piece_degrees))

    corners = []
    for quarter in range(4):
        _, peak_radius, peak_degrees = min(peaks.get(quarter, [(0, 0, 0)]))
        peak_radians = math.radians(peak_degrees)
        dx = peak_radius * math.cos(peak_radians)
        dy = peak_radius * math.sin(peak_radians)
        corners.append((int(cx+dx), int(cy+dy)))
    all_corners.append(corners)
    
    cv2.line(img, corners[0], corners[2], (255, 0, 0), 3)
    cv2.line(img, corners[1], corners[3], (255, 0, 0), 3)

plt.imshow(img);

In [ ]:
img = img_orig.copy()
all_edges_contours = []  # list of [contour]

def length(v):
    return v[0]**2 + v[1]**2

def sub_contour(c, idx0, idx1):
    if idx1 > idx0:
        return c[idx0:idx1]
    else:
        return np.concatenate([c[idx0:], c[:idx1]])

for contour, corners in zip(contours, all_corners):
    indices = []
    for corner in corners:
        distances = [length(p[0] - corner) for p in contour]
        idx = distances.index(min(distances))
        indices.append(idx)

    edges_contours = [
        sub_contour(contour, indices[0], indices[3]),
        sub_contour(contour, indices[1], indices[0]),
        sub_contour(contour, indices[2], indices[1]),
        sub_contour(contour, indices[3], indices[2])
    ]
    all_edges_contours.append(edges_contours)

    cv2.drawContours(img, edges_contours[0], -1, (0, 255, 0), 6)
    cv2.drawContours(img, edges_contours[1], -1, (255, 0, 0), 6)
    cv2.drawContours(img, edges_contours[2], -1, (0, 255, 0), 6)
    cv2.drawContours(img, edges_contours[3], -1, (255, 0, 0), 6)

plt.imshow(img);

In [ ]:
all_edges_straight_lengths = []  # list of double
all_edges_arc_lengths = []  # list of double
all_edges_angles = []  # list of degrees
all_edges_heights = []  # list of double
all_edges_types = []  # list of 'flat', 'male', 'female'

img = img_orig.copy()

for edges_contours in all_edges_contours:
    arc_lengths = []
    straight_lengths = []
    heights = []
    angles = []
    types = []
    for contour in edges_contours:
        p0 = contour[0][0]
        p1 = contour[-1][0]
        dx, dy = p1 - p0
        arc_lengths.append(cv2.arcLength(contour, closed=False))
        straight_lengths.append(math.sqrt(dx**2 + dy**2))
        radians = math.atan2(dy, dx)
        angles.append(math.degrees(radians))
        
        sin = math.sin(radians)
        cos = math.cos(radians)
        matrix = np.array([[cos, -sin], [sin, cos]])
        normalized_points = (contour - p0) @ matrix  # first point at (0, 0), last point at (X, 0)
        heights = normalized_points[:,0,1]
        min_height = min(heights)
        max_height = max(heights)
        if abs(max_height) + abs(min_height) < 10:
            edge_type = "flat"
            color = (0, 255, 0)
        elif abs(max_height) > abs(min_height):
            edge_type = "male"
            color = (255, 0, 0)
        else:
            edge_type = "female"
            color = (0, 0, 255)
        types.append(edge_type)
        cv2.drawContours(img, contour, -1, color, 6)

    all_edges_arc_lengths.append(arc_lengths)
    all_edges_straight_lengths.append(straight_lengths)
    all_edges_angles.append(angles)
    all_edges_heights.append(heights)
    all_edges_types.append(types)

plt.imshow(img)

In [ ]:
ipyplot.plot_class_tabs(images=[piece.img_orig for piece in pieces], labels=[piece.nb_flats for piece in pieces], img_width=50)

In [ ]:
nb_flats = Counter([piece.nb_flats for piece in pieces])

assert nb_flats[2] == 4
# H**2 - H*B/2 + I = 0
a = 1
b = - nb_flats[1] / 2
c = nb_flats[0]
delta = b**2 - 4*a*c
inner_height = int((-b - math.sqrt(delta)) / (2*a))
inner_width = int((-b + math.sqrt(delta)) / (2*a))
print(f"Size of puzzle: {2 + inner_width}x{2 + inner_height}")
assert inner_height * inner_width == nb_flats[0]
assert 2 * (inner_height + inner_width) == nb_flats[1]

In [ ]:
outer_pieces = [piece.rotate_border() for piece in pieces if piece.nb_flats > 0]
ipyplot.plot_images(images=[piece.img_orig for piece in outer_pieces], img_width=50)
